In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tf_explain.callbacks.activations_visualization import ActivationsVisualizationCallback

In [2]:
!wget -c https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!mkdir -p ${HOME}/.keras/datasets
!tar -xzvf cifar-10-python.tar.gz -C ${HOME}/.keras/datasets

--2020-09-13 18:03:06--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [2]:
# Model configuration
batch_size = 32
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 10
no_epochs = 10
optimizer = Adam()
validation_split = 0.2
verbosity = 1

In [3]:
# Load CIFAR-10 data
(input_train, target_train), (input_test, target_test) = cifar10.load_data()
# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)
# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')
# Scale data
input_train = input_train / 255
input_test = input_test / 255

In [4]:
# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', name='visualization_layer'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

In [5]:
# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

In [6]:
# Define the Activation Visualization callback
output_dir = './visualizations'
callbacks = [
    ActivationsVisualizationCallback(
        validation_data=(input_test, target_test),
        layers_name=['visualization_layer'],
        output_dir=output_dir,
    ),
]

In [7]:
# Fit data to model
history = model.fit(input_train, target_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split,
            callbacks=callbacks)

Epoch 1/10
1250/1250 [==============================] - 84s 67ms/step - loss: 1.3805 - accuracy: 0.5038 - val_loss: 1.0705 - val_accuracy: 0.6225
Epoch 2/10
1250/1250 [==============================] - 81s 65ms/step - loss: 0.9351 - accuracy: 0.6723 - val_loss: 0.9555 - val_accuracy: 0.6631
Epoch 3/10
1250/1250 [==============================] - 81s 65ms/step - loss: 0.6489 - accuracy: 0.7743 - val_loss: 0.9866 - val_accuracy: 0.6625
Epoch 4/10
1250/1250 [==============================] - 84s 67ms/step - loss: 0.3675 - accuracy: 0.8720 - val_loss: 1.2476 - val_accuracy: 0.6522
Epoch 5/10
1250/1250 [==============================] - 85s 68ms/step - loss: 0.1725 - accuracy: 0.9416 - val_loss: 1.5888 - val_accuracy: 0.6436
Epoch 6/10
1250/1250 [==============================] - 84s 67ms/step - loss: 0.1121 - accuracy: 0.9621 - val_loss: 2.0108 - val_accuracy: 0.6413
Epoch 7/10
1250/1250 [==============================] - 84s 67ms/step - loss: 0.0868 - accuracy: 0.9699 - val_loss: 2.0655 -

In [9]:
# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 2.5153918266296387 / Test accuracy: 0.630299985408783
